In [44]:
import os
import time

# spark imports
import findspark
import pickle
findspark.init('C:/Users/Lenovo/Downloads/spark-3.0.0-preview2-bin-hadoop3.2')

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

from pyspark.sql.functions import col

from pyspark.mllib.recommendation import ALS

In [2]:
sc = SparkContext('local')
spark = SparkSession(sc)

In [3]:
books = spark.read.load(r"C:\Users\Lenovo\Documents\Springboard\Capstone\Dataset\BX-CSV-Dump\BX-Books.csv",
                     format="csv", sep=";", inferSchema="true", header="true", encoding ='latin1')

In [4]:
win_row_number = Window.orderBy("ISBN")
isbn = books.select(F.row_number().over(win_row_number).alias("book_id"), "ISBN")
isbn.cache().show(3)

+-------+----------+
|book_id|      ISBN|
+-------+----------+
|      1|0000913154|
|      2|0001010565|
|      3|0001046438|
+-------+----------+
only showing top 3 rows



In [5]:
ratings = spark.read.load(r"C:\Users\Lenovo\Documents\Springboard\Capstone\Dataset\BX-CSV-Dump\BX-Book-Ratings.csv",
                     format="csv", sep=";", inferSchema="true", header="true", encoding ='latin1')

In [13]:
ratings = ratings.withColumn("Book-Rating", ratings["Book-Rating"].cast("Float"))
ratings_with_bid = ratings.join(isbn, on=["ISBN"], how="inner").select(col("User-ID").alias("userID"), col("book_id").alias("productID"), col("Book-Rating").alias("rating"))
ratings_with_bid.show(3)

+------+---------+------+
|userID|productID|rating|
+------+---------+------+
|276725|    46116|   0.0|
|276726|    22830|   5.0|
|276727|    92994|   0.0|
+------+---------+------+
only showing top 3 rows



In [14]:
sample_ratings_with_bid = ratings_with_bid.sample(False, 0.0001, 3)
sample_ratings_with_bid.show(3)

+------+---------+------+
|userID|productID|rating|
+------+---------+------+
|277107|    62084|  10.0|
|277523|   109110|  10.0|
|  1172|   257519|  10.0|
+------+---------+------+
only showing top 3 rows



In [27]:
model = ALS.train(sample_ratings_with_bid, rank=5, iterations=5, lambda_=0.01, blocks=-1, nonnegative=False, seed=None)

In [30]:
model.recommendProducts(1172, 10)

[Rating(user=1172, product=257519, rating=9.998900549418968),
 Rating(user=1172, product=40024, rating=6.675842412892367),
 Rating(user=1172, product=89231, rating=3.5002362975976222),
 Rating(user=1172, product=138582, rating=3.1065279285674716),
 Rating(user=1172, product=62084, rating=3.095510764265299),
 Rating(user=1172, product=8452, rating=2.9277114709539447),
 Rating(user=1172, product=159196, rating=2.8056055670813453),
 Rating(user=1172, product=70987, rating=2.362546361365366),
 Rating(user=1172, product=253763, rating=2.3097938579809707),
 Rating(user=1172, product=123133, rating=2.2159410985995827)]

In [46]:
# save the model to disk
filename = r'C:\Users\Lenovo\Documents\Springboard\Capstone\Model\ALS\model.sav'
pickle.dump(model, open(filename, 'wb'))

Exception: It appears that you are attempting to reference SparkContext from a broadcast variable, action, or transformation. SparkContext can only be used on the driver, not in code that it run on workers. For more information, see SPARK-5063.